In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.graph_objs as go
import plotly
from plotly import tools
import plotly.express as px
from scipy.stats import boxcox
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 100)

In [2]:
'''https://www.kaggle.com/agirlcoding/all-space-missions-from-1957'''
'''https://nextspaceflight.com/about/'''

'https://nextspaceflight.com/about/'

In [3]:
mission = pd.read_csv(r'C:\Users\user\Desktop\Space_Corrected.csv')
mission

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
...,...,...,...,...,...,...,...,...,...
4319,4319,4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure
4320,4320,4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success
4321,4321,4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Fri Dec 06, 1957 16:44 UTC",Vanguard | Vanguard TV3,StatusRetired,NaN,Failure
4322,4322,4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan","Sun Nov 03, 1957 02:30 UTC",Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success


In [4]:
mission.shape

(4324, 9)

#### Finding out how many vales are missing from the data frame.

In [5]:
mission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
Unnamed: 0        4324 non-null int64
Unnamed: 0.1      4324 non-null int64
Company Name      4324 non-null object
Location          4324 non-null object
Datum             4324 non-null object
Detail            4324 non-null object
Status Rocket     4324 non-null object
 Rocket           964 non-null object
Status Mission    4324 non-null object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


In [6]:
mission.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Company Name', 'Location', 'Datum',
       'Detail', 'Status Rocket', ' Rocket', 'Status Mission'],
      dtype='object')

#### Both columns have the same values due to lack of value for this project its remeoved.

In [7]:
mission = mission.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

#### Renamed columns to better understand the data.

In [8]:
mission.rename(columns={' Rocket' : 'Cost of the mission','Detal' : 'Name of the rocket', 'Detail' : 'Rocket Name'}, inplace=True)

In [9]:
mission.columns

Index(['Company Name', 'Location', 'Datum', 'Rocket Name', 'Status Rocket',
       'Cost of the mission', 'Status Mission'],
      dtype='object')

#### Removed a one duplicated mission, to have clearity from noise.

In [10]:
duplicated = mission[mission.duplicated() == True]
duplicated

,Company Name,Location,Datum,Rocket Name,Status Rocket,Cost of the mission,Status Mission
793,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Wed Nov 05, 2008 00:15 UTC",Long March 2D | Shiyan-3 & Chuangxin-1(02),StatusActive,29.75,Success


In [11]:
mission = mission.drop_duplicates()

In [12]:
mission.shape

(4323, 7)

#### The procentage of missing values, which is dominating only one column. Maybe isn't offical information about cost of mission is realising to public due to keep safty.

In [13]:
mission.isnull().sum()/len(mission) * 100

Company Name            0.000000
Location                0.000000
Datum                   0.000000
Rocket Name             0.000000
Status Rocket           0.000000
Cost of the mission    77.723803
Status Mission          0.000000
dtype: float64

#### Basic statistic/distribution about data.

In [14]:
mission.describe()

,Company Name,Location,Datum,Rocket Name,Status Rocket,Cost of the mission,Status Mission
count,4323,4323,4323,4323,4323,963,4323
unique,56,137,4319,4278,2,56,4
top,RVSN USSR,"Site 31/6, Baikonur Cosmodrome, Kazakhstan","Sun Aug 25, 1991 08:40 UTC",Cosmos-3MRB (65MRB) | BOR-5 Shuttle,StatusRetired,450.0,Success
freq,1777,235,2,6,3534,136,3878


In [15]:
mission['Cost of the mission'] = mission['Cost of the mission'].str.replace(',','').astype(float)

In [16]:
mission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4323 entries, 0 to 4323
Data columns (total 7 columns):
Company Name           4323 non-null object
Location               4323 non-null object
Datum                  4323 non-null object
Rocket Name            4323 non-null object
Status Rocket          4323 non-null object
Cost of the mission    963 non-null float64
Status Mission         4323 non-null object
dtypes: float64(1), object(6)
memory usage: 270.2+ KB


#### Extracting values from a columns, to manage them in further analyses.

In [17]:
mission['Year'] = pd.to_datetime(mission['Datum']).apply(lambda x: x.year)
mission['Month'] = pd.to_datetime(mission['Datum']).apply(lambda x: x.month)
mission['Day'] = pd.to_datetime(mission['Datum']).apply(lambda x: x.day)

#### Take only country name to see some trends.

In [18]:
mission["Country"] = mission["Location"].apply(lambda x: x.strip().split(", ")[-1])

In [19]:
mission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4323 entries, 0 to 4323
Data columns (total 11 columns):
Company Name           4323 non-null object
Location               4323 non-null object
Datum                  4323 non-null object
Rocket Name            4323 non-null object
Status Rocket          4323 non-null object
Cost of the mission    963 non-null float64
Status Mission         4323 non-null object
Year                   4323 non-null int64
Month                  4323 non-null int64
Day                    4323 non-null int64
Country                4323 non-null object
dtypes: float64(1), int64(3), object(7)
memory usage: 405.3+ KB


#### Ploting countries which have participate in expoloring space.

In [20]:
no_lauches = mission["Country"].value_counts()
no_lauches.iplot(kind='bar', xTitle = 'Country', yTitle = "Count", title = 'Number of launches by Country', color = 'blue')

#### There are few big fish in this area. Russia and Kazakhstan are related due to history that show the bouth belong to the same country, stands for 'ZSRR'. From history we know about big competion in expoloring cosmos and landing on the moon. So it's not a surprised to notice USA. Then we have France and next China.